In [1]:
#/* *************************************************************************** */
#/*                                                                             */
#/*   XXXVIII PhD Course on Big Data & AI @ Mercatorum University               */
#/*                                                                             */
#/*   By: vstile <vittoriostile@gmail.com>                                      */
#/*   Link-in-bio page: https://linktr.ee/vstile                                */
#/*                                                                             */
#/*   Created: 2024/11/21 by vstile                                             */
#/*   Updated: 2025/03/24 by vstile                                             */
#/*                                                                             */
#/* *************************************************************************** */

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This implementation handles REAL videos and FAKE videos across five manipulation methods. The model uses VGG16 for feature extraction and a custom classifier for binary classification (REAL vs. FAKE).

1. Loading the dataset from the specified paths.
2. Preprocessing frames to extract faces.
3. Training the model using VGG16.
4. Calculating and displaying the accuracy of the model.

# 1. Define Paths and Labels

In [3]:
# # Paths to REAL and FAKE videos
# REAL_VIDEOS_PATH = "/Volumes/Macintosh SSD/Developer/DeepFace_Detect/original_sequences/youtube/c40/videos"
# FAKE_VIDEOS_PATHS = [
#     "/Volumes/Macintosh SSD/Developer/DeepFace_Detect/manipulated_sequences/Deepfakes/c40/videos",
#     "/Volumes/Macintosh SSD/Developer/DeepFace_Detect/manipulated_sequences/Face2Face/c40/videos",
#     "/Volumes/Macintosh SSD/Developer/DeepFace_Detect/manipulated_sequences/FaceShifter/c40/videos",
#     "/Volumes/Macintosh SSD/Developer/DeepFace_Detect/manipulated_sequences/FaceSwap/c40/videos",
#     "/Volumes/Macintosh SSD/Developer/DeepFace_Detect/manipulated_sequences/NeuralTextures/c40/videos",
# ]

# LABELS = {"REAL": 0, "FAKE": 1}


La stringa nella lista **REAL_VIDEOS_PATH** punta alla directory che contiene video REAL non manipolati.

Ogni stringa nella lista **FAKE_VIDEOS_PATHS** rappresenta il percorso di una cartella specifica per una delle 5 tecniche di manipolazione (Deepfakes, Face2Face, ecc.) che contengono ognuna video FAKE manipolati:
* **Deepfakes**: Manipolazione basata su GAN (reti generative avversarie o generative adversarial network).
* **Face2Face**: Tecnica di manipolazione che sostituisce espressioni facciali in tempo reale.
* **FaceShifter**: Tecnica avanzata di scambio di volti.
* **FaceSwap**: Manipolazione che sostituisce l'intero volto con quello di un altro individuo.
* **NeuralTextures**: Tecnica che manipola i dettagli della texture del volto.

**REAL_VIDEOS_PATHS** e **FAKE_VIDEOS_PATHS** saranno passati come input alla funzione **load_dataset** o **load_subset_dataset** (definita più avanti)

# 2. Functions for Data Extraction and Preprocessing


In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

Riepilogo del Flusso:
1. Apre il video e legge i frame in un ciclo.
2. Salta i frame in base a frame_skip per ottimizzare la velocità.
3. Ridimensiona e converte in scala di grigi i frame selezionati.
4. Utilizza Haar Cascade per rilevare i volti.
5. Estrae, ridimensiona e memorizza i volti insieme alle etichettee.
6. Restituisce i volti estratti e le etichette.

In [5]:
# # Function to extract frames and detect faces from a SUBSET
# def extract_faces_from_video(video_path, label, cascade_classifier, frame_skip=30): #Qui ho messo 30 per velocizzare il calcolo, di solito si usa 10
#     faces = [] # Inizializzo lista vuota per momorizzare i frame
#     labels = [] # Inizializzo lista vuota per le etichette REAL e FAKE
#     video_origins = []  # Nuova lista per tracciare i video di origine
#     cap = cv2.VideoCapture(video_path) # Oggetto VideoCapture di OpenCV, utilizzato per aprire e leggere il video specificato in video_path
#     frame_count = 0 # Contatore per tracciare il numero di frame elaborati

# # Ciclo per Elaborare il Video
#     while cap.isOpened(): # cap.isOpened(): Verifica se il video è aperto e leggibile
#         ret, frame = cap.read() # cap.read(): Legge un frame dal video
#         if not ret: # ret: Valore booleano che indica se il frame è stato letto con successo
#             break # Se invece il frame non può essere letto (es. fine del video), il ciclo viene interrotto

# # Rilevamento dei volti con Haar Cascades
#         if frame_count % frame_skip == 0: # Elabora un frame ogni frame_skip (es. ogni 10 frame, se frame_skip=10)
#             frame = cv2.resize(frame, (640, 480))  # Ridimensiona il frame per velocizzare
#             gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Converte il frame in scala di grigi per facilitare il classificatori Haar Cascade
#             faces_detected = cascade_classifier.detectMultiScale(
#                 gray, scaleFactor=1.3, minNeighbors=5
#             )

#            # Elaborazione dei Volti Rilevati
#             for (x, y, w, h) in faces_detected: # Per ogni volto rilevato
#                 face = frame[y:y+h, x:x+w] # Viene estratta la regione del volto dal frame originale usando le coordinate rilevate
#                 face_resized = cv2.resize(face, (224, 224))  # Ridimensiona a 224x224
#                 faces.append(face_resized) # Aggiunge il volto estratto e ridimensionato alla lista faces
#                 labels.append(label) # Aggiunge l'etichetta (REAL o FAKE) corrispondente al volto estratto
#                 video_origins.append(video_path)  # Salva il percorso del video di origine

#         frame_count += 1 # Incrementa il contatore dei frame (basato su frame_skip)

#     cap.release() # Rilascia le risorse associate all'oggetto VideoCapture dopo che il video è stato completamente elaborato
#     return np.array(faces), np.array(labels), video_origins # In output la lista dei volti estratti e lista delle etichette associate e i percorsi dei video



In [6]:

# # Function to load the dataset
# def load_subset_dataset(real_path, fake_paths, cascade_path, max_videos=10): # Qui aggiungo parametro max_videos per ridurre il numero di video analizzati per ogni directory
#     face_cascade = cv2.CascadeClassifier(cascade_path)  # Carica il classificatore Haar

#     faces = []
#     labels = []
#     origins = []  # Lista per tracciare le origini dei frame


#     # Carica un sottoinsieme di video REAL
#     for video in os.listdir(real_path)[:max_videos]:
#         video_path = os.path.join(real_path, video)
#         face_frames, face_labels, video_origins = extract_faces_from_video(
#             video_path, LABELS["REAL"], face_cascade
#         ) # Ogni volto estratto dai video in queste directory viene associato all'etichetta LABELS["REAL"], ovvero 0
#         faces.extend(face_frames)
#         labels.extend(face_labels)
#         origins.extend(video_origins)

#     # Carica un sottoinsieme di video FAKE
#     for fake_path in fake_paths: #la lista fake_paths (che punta a FAKE_VIDEOS_PATHS) viene iterata per ogni sottocartella (Deepfakes/c40/videos; Face2Face/c40/videos etc...)
#         for video in os.listdir(fake_path)[:max_videos]:
#             video_path = os.path.join(fake_path, video)
#             face_frames, face_labels, video_origins = extract_faces_from_video(
#                 video_path, LABELS["FAKE"], face_cascade
#             ) # Ogni volto estratto dai video in queste directory viene associato all'etichetta LABELS["FAKE"], ovvero 1
#             faces.extend(face_frames)
#             labels.extend(face_labels)
#             origins.extend(video_origins)

#     return np.array(faces), np.array(labels), origins #array faces con le immagini dei volti e array labels con le etichette numeriche corrispondenti (0 per REAL, 1 per FAKE)


In [7]:

# # Load the dataset

# # Definizione del percorso del file per il classificatore Haar Cascade
# cascade_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"

# # Esegui la funzione per iterare su tutte le directory e assocuare etichette REAL e FAKE con il percorso del file Haar Cascade
# faces, labels, origins = load_subset_dataset(REAL_VIDEOS_PATH, FAKE_VIDEOS_PATHS, cascade_path)

# # REAL_VIDEOS_PATH: Percorso per i video REAL.
# # FAKE_VIDEOS_PATHS: Lista di percorsi alle directory dei video FAKE.
# # cascade_path: File Haar Cascade per il rilevamento dei volti.

Rilevamento dei volti con **Haar Cascades**

Per il rilevamento dei volti usiamo Haar Cascades (principalmente perché non usa GPU)

**cascade_classifier.detectMultiScale** Rileva i volti nel frame in scala di grigi utilizzando Haar Cascades.

*Parametri:*
* scaleFactor=1.3: Indica di quanto viene ridotta la dimensione del frame ad ogni scala per il rilevamento.
* minNeighbors=5: Numero minimo di rettangoli vicini richiesti per considerare una regione come volto.

*Output:*
* Un array di coordinate (x, y, w, h) per ciascun volto rilevato.

# 3. Split Dataset into Training, Validation, and Test Sets

Riepilogo del Flusso:
1. Normalizzazione: Convertiamo i valori RGB in un intervallo tra 0 e 1 per stabilità e uniformità.
2. Divisione Set: I dati vengono suddivisi in training, validation e test per addestrare, monitorare e valutare il modello.
3. Stratificazione: Si mantiene la proporzione delle classi (REAL/FAKE) uniforme tra i diversi set, evitando squilibri.

In [8]:
# # Normalize RGB values
# faces = faces / 255.0

# # Split dataset 1
# X_train, X_temp, y_train, y_temp, origins_train, origins_temp  = train_test_split(faces, labels, origins, test_size=0.28, stratify=labels, random_state=42)

# # Split dataset 2
# X_val, X_test, y_val, y_test, origins_val, origins_test = train_test_split(X_temp, y_temp, origins_temp, test_size=0.5, stratify=y_temp, random_state=42)

# print(f"Training set size: {len(X_train)}")
# print(f"Validation set size: {len(X_val)}")
# print(f"Test set size: {len(X_test)}")


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Percorsi delle directory contenenti le facce estratte dai video FAKE e REAL
FAKE_FACES_PATH = "/content/drive/MyDrive/Faces_skip_20/FAKE_faces"
REAL_FACES_PATH = "/content/drive/MyDrive/Faces_skip_20/REAL_faces"

# Etichette per i video REAL e FAKE
LABELS = {"REAL": 0, "FAKE": 1}

# Inizializza le liste per le facce, le etichette e le origini
faces = []
labels = []
origins = []

# Funzione per caricare le immagini da una directory e assegnare le etichette
def load_faces_from_directory(directory, label, origin_label):
    count = 0
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg"):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path)
            if img is not None:
                faces.append(img)
                labels.append(label)
                origins.append(origin_label)
                count += 1
    print(f"Caricate {count} immagini dalla directory {directory}")

# Carica le facce dai video FAKE
print("Inizio caricamento facce dai video FAKE")
load_faces_from_directory(FAKE_FACES_PATH, LABELS["FAKE"], "FAKE")

# Carica le facce dai video REAL
print("Inizio caricamento facce dai video REAL")
load_faces_from_directory(REAL_FACES_PATH, LABELS["REAL"], "REAL")

# Converti le liste in array numpy
print("Converti delle liste in array numpy")
faces = np.array(faces)
labels = np.array(labels)
origins = np.array(origins)
print("Conversione effettuata delle liste in array numpy")

# Normalize RGB values
print("Inizio normalizzazione di faces")
faces = faces / 255.0
print("Normalizzazione effettuata correttamente per faces")

# Stampa il numero totale di facce caricate
print(f"Totale facce caricate: {len(faces)}")
print(f"Totale etichette caricate: {len(labels)}")
print(f"Totale origini caricate: {len(origins)}")

# Split del dataset in training, validation e test set
print("Inizio split del dataset")
X_train, X_temp, y_train, y_temp, origins_train, origins_temp = train_test_split(faces, labels, origins, test_size=0.28, random_state=42)
X_val, X_test, y_val, y_test, origins_val, origins_test = train_test_split(X_temp, y_temp, origins_temp, test_size=0.5, random_state=42)
print("Split del dataset completato")

# Stampa delle dimensioni dei dataset dopo lo split
print(f"Training set: {len(X_train)} samples")
print(f"Validation set: {len(X_val)} samples")
print(f"Test set: {len(X_test)} samples")

Inizio caricamento facce dai video FAKE


**Normalize RGB values**
Converte i valori dei pixel da un intervallo [0, 255] a un intervallo [0, 1].
I pixel nelle immagini digitali RGB hanno valori compresi tra 0 (nero) e 255 (bianco). Dividendo tutti i valori per 255, otteniamo una scala normalizzata tra 0 e 1. Per varie ragioni nei modelli di deep learning funzionano meglio con valori normalizzati.

**Split dataset 1**

**train_test_split**: Una funzione di scikit-learn utilizzata per dividere il dataset in modo casuale.

*Parametri:*
* faces: Dati delle immagini (features).
* labels: Etichette associate (REAL o FAKE).
* test_size=0.28: Il 28% dei dati totali viene assegnato al set temporaneo X_temp.
* stratify=labels: Assicura che la proporzione di etichette (REAL/FAKE) sia mantenuta costante tra training e il resto.
* random_state=42: Seed per la riproducibilità dei risultati.

*Output:*
* X_train: Immagini per il training.
* y_train: Etichette per il training.
* X_temp: Immagini temporanee per essere ulteriormente divise.
* y_temp: Etichette temporanee.

**Split dataset 2**

Questa divisione prende il set temporaneo (28%) e lo divide equamente in:
1. Validation Set 14%: Che serve per monitorare le prestazioni del modello durante il training. Utilizzato per determinare quando il modello inizia a sovradattarsi (overfitting).
2. Test Set 14%: Che serve per valutare le prestazioni finali del modello dopo l'addestramento. Non viene utilizzato durante il training per evitare bias.

*Parametri:*
* X_temp e y_temp: Immagini ed etichette temporanee.
* test_size=0.5: Divide il set temporaneo equamente tra validation e test.
* stratify=y_temp: Mantiene la distribuzione di etichette (REAL/FAKE) in output dalla precedente.
* random_state=42: Stesso seed utilizzato per la riproducibilità dei risultati.

**Risultati Attesi**
Supponendo che ci siano 859 immagini totali:

* Training Set: 72% → 618 immagini.
* Validation Set: 14% → 120 immagini.
* Test Set: 14% → 121 immagini.

# 4. Build and Compile VGG16 Model

Riepilogo del Flusso:
1. Caricare il Modello VGG16 Pre-Addestrato
2. Congelare i Livelli del Modello di Base
3. Aggiungere Livelli di Classificazione Personalizzati
4. Creare il Modello Completo
5. Compilare il Modello
6. Mostrare in output un Sommario del Modello

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Carica il modello pre-addestrato VGG16 senza il top
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congela i pesi del modello pre-addestrato
for layer in base_model.layers:
    layer.trainable = False

# Aggiungi i tuoi livelli di classificazione
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

# Crea il modello
model = Model(inputs=base_model.input, outputs=x)

# Compila il modello
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Addestra il modello
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), batch_size=32)

# Valuta il modello
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")

**Step 4: Creare il Modello Completo**

**Funzione: model**:
* inputs=base_model.input: L'input del modello è lo stesso della ResNet50 pre-addestrata (immagini 224x224x3).
* outputs=output: L'output finale è il livello di classificazione binaria personalizzato.

**Step 5: Compilare il Modello**

**Ottimizzatore: optimizer='adam'**
* Utilizza l'algoritmo di ottimizzazione Adam, che combina i vantaggi di RMSprop e Stochastic Gradient Descent (SGD).
* Efficiente per problemi complessi e richiede poca regolazione manuale.

**Funzione di Perdita: loss='binary_crossentropy'**
* Adatta ai problemi di classificazione binaria.
* Misura la differenza tra le probabilità previste dal modello (valori sigmoidi tra 0 e 1) e le etichette binarie reali (0 o 1).

**Metriche di Valutazione: metrics=['accuracy']**
* Valuta il modello durante il training e la validazione basandosi sull'accuratezza.

**Step 6: Mostrare un Sommario del Modello model.summary()**

Stampa una descrizione dettagliata del modello:
* Architettura.
* Numero di parametri (trainabili e non trainabili).
* Livelli aggiunti.

# 5. Visualize Training Results

Riepilogo del Flusso:
1. Tracciare l'accuratezza in % (accuracy)
2. Tracciare la perdita (loss)

Questi valori sono memorizzati nell'oggetto history, che è il risultato di model.fit.


In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy') # history.history['accuracy']: Contiene i valori di accuratezza calcolati dal modello su ogni epoca per il training set
plt.plot(history.history['val_accuracy'], label='Validation Accuracy') # history.history['val_accuracy']: valori calcolati sul validation set dopo ogni epoca, ci dice quanto bene il modello generalizza su dati non visti durante il training
plt.legend() # Stampa la leggenda in basso a dx
plt.title('Accuracy') # Da il titolo al grafico in alto
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Train Loss') # history.history['loss']: i valori della funzione di perdita calcolati dal modello sul training set durante ogni epoca
plt.plot(history.history['val_loss'], label='Validation Loss') # history.history['val_loss']: i valori calcolati sul validation set dopo ogni epoca, ci dice quanto bene il modello generalizza su dati non visti
plt.legend()
plt.title('Loss')
plt.show()


# 6. Identify FAKE videos

Per identificare i video di origine dei frame dichiarati FAKE in fase di testing, è stata tracciata l'associazione tra ciascun frame e il video da cui è stato estratto tramite un dizionario che memorizza l'origine dei frame, così da poter analizzare i risultati dei test.

In [ ]:
# Test del modello
predictions = model.predict(X_test)  # Probabilità per ogni frame
predicted_labels = (predictions > 0.79).astype(int)  # Soglia: 0.79

# Trova i frame predetti come FAKE
fake_indices = np.where(predicted_labels == 1)[0]  # Indici dei frame FAKE

# Identifica i video di origine per i frame FAKE
fake_videos = [origins_test[i] for i in fake_indices]

print(f"Numero di frame FAKE identificati: {len(fake_indices)}")
print("Video di origine per i frame FAKE:")
for video in set(fake_videos):  # Usa set() per evitare duplicati
    print(video)
